In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/initial-arrangment/initial_arangment.pkl
/kaggle/input/distance/Distance_Matrix.csv
/kaggle/input/biomass-data/biomass.csv
/kaggle/input/optimizer1/keras/default/1/shell_v1_optimizer_agent.h5
/kaggle/input/biomass/Biomass_History.csv


In [2]:
!pip install keras-rl2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.6 MB/s eta 0:00:00


In [3]:
import plotly.express as px
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import MeanSquaredLogarithmicError,MeanSquaredError,MeanAbsoluteError
from matplotlib import pyplot as plt
from shapely.geometry import Point, MultiPoint
import rtree
from sklearn.neighbors import NearestNeighbors
from gym import Env
from gym.spaces import Box, MultiDiscrete, Dict,Discrete
from shapely.ops import nearest_points
import geopandas as gpd
import seaborn as sns
import random
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import bisect
import copy
import pickle
import pandas as pd
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# Data Entry and Point Creation


In [4]:
biomass = pd.read_csv("/kaggle/input/biomass/Biomass_History.csv")
distance = pd.read_csv("/kaggle/input/distance/Distance_Matrix.csv")
# creating point objects, will be useful later on 
points = []
for i, row in biomass.iterrows():
    points.append((float(row['Longitude']),float(row["Latitude"])) )
points2 = copy.deepcopy(points)
biomass["Geometries"] = points
biomass["Type"] = "Biomass prod.plants"
print(biomass.head())


   Index  Latitude  Longitude       2010       2011       2012       2013  \
0      0  24.66818   71.33144   8.475744   8.868568   9.202181   6.023070   
1      1  24.66818   71.41106  24.029778  28.551348  25.866415  21.634459   
2      2  24.66818   71.49069  44.831635  66.111168  56.982258  53.003735   
3      3  24.66818   71.57031  59.974419  80.821304  78.956543  63.160561   
4      4  24.66818   71.64994  14.653370  19.327524  21.928144  17.899586   

        2014       2015       2016        2017            Geometries  \
0  10.788374   6.647325   7.387925    5.180296  (71.33144, 24.66818)   
1  34.419411  27.361908  40.431847   42.126945  (71.41106, 24.66818)   
2  70.917908  42.517117  59.181629   73.203232  (71.49069, 24.66818)   
3  93.513924  70.203171  74.536720  101.067352  (71.57031, 24.66818)   
4  19.534035  19.165791  16.531315   26.086885  (71.64994, 24.66818)   

                  Type  
0  Biomass prod.plants  
1  Biomass prod.plants  
2  Biomass prod.plants  
3  B

In [ ]:
sns.heatmap(biomass.corr(), annot=True)
plt.show()

In [5]:
fig = px.scatter_mapbox(biomass, lat="Latitude", lon="Longitude",hover_data=["Index"],
                        color="2017", zoom=3, height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show(renderer='iframe')

In [ ]:
biomass["2017"].hist(bins = 20)
plt.show()

In [6]:
import plotly.express as px
import pandas as pd

def visualize_depots_refineries_px(depots, refineries, points):
    # Prepare the data
    data = []
    
    # Loop through depots and their sources
    for i, depot in enumerate(depots):
        try:
            depot_index = depot.index
            depot_sources = depot.sources_index
            
            # Depot itself
            if depot_index is not None:
                #print(points[depot_index])
                data.append({
                    'Type': f"Depot {i}",
                    'Lat': points[depot_index][1],
                    'Lon': points[depot_index][0],
                    'Color': f"Depot {i}",
                    'Marker': 'square'
                })

            # Sources of the depot
            for source_index in depot_sources:
                data.append({
                    'Type': f"Depot {i} Source",
                    'Lat': points[source_index][1],
                    'Lon': points[source_index][0],
                    'Color': f"Depot {i}",
                    'Marker': 'circle'
                })
        except Exception as e:
            print(f"Error processing depot {i}: {e}")
            continue

    # Loop through refineries
    for i, refinery in enumerate(refineries):
        try:
            refinery_index = refinery.index
            if refinery_index is not None:
                data.append({
                    'Type': f"Refinery {i}",
                    'Lat': points[refinery_index][1],
                    'Lon': points[refinery_index][0],
                    'Color': 'Refinery',
                    'Marker': 'cross'
                })
        except Exception as e:
            print(f"Error processing refinery {i}: {e}")
            continue

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)

    # Create the scatter plot using Plotly Express
    fig = px.scatter_mapbox(df,
                            lat='Lat',
                            lon='Lon',
                            color='Color',
                            #symbol='Marker',
                            hover_name='Type',
                            zoom=5,
                            mapbox_style="open-street-map")
    
    # Show the plot
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show(renderer="iframe")





In [ ]:
count, bins_count = np.histogram(biomass["2017"], bins=20) 
  
# finding the PDF of the histogram using count values 
pdf = count / sum(count) 
  
# using numpy np.cumsum to calculate the CDF 
# We can also find using the PDF values by looping and adding 
cdf = np.cumsum(pdf) 
  
# plotting PDF and CDF 
plt.plot(bins_count[1:], pdf, color="red", label="PDF") 
plt.plot(bins_count[1:], cdf, label="CDF") 

plt.legend() 
plt.show()

In [ ]:
#predictions using neural networks

""" First a little data prep. We need to batch the results from 2010 till 2016, the 2017 
results will serve as the target values for the neural network"""

x = biomass[[str(i+2000) for i in range(10,17)]].values
print(x.shape)
y = biomass["2017"].values
print(y.shape)
scaler = StandardScaler()
x = scaler.fit_transform(x)
X_train,X_test,Y_train,Y_test = train_test_split(x,y, test_size = 0.25, random_state = 32)
print (X_train.shape, X_test.shape)


def build_net_no_dropout(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(input_size,activation = 'relu'))
    model.add(tf.keras.layers.Dense(128, activation = "relu"))
    model.add(tf.keras.layers.Dense(32, activation = "relu"))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    return model


def build_net_with_dropout(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(input_size,activation = 'relu',kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(128, activation = "relu",kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dropout(0.7))
    model.add(tf.keras.layers.Dense(32, activation = "relu",kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    return model

def model_run(model,train_x,val_y, train_y, val_x, num_epochs):
    loss = MeanSquaredLogarithmicError()
    optimizer = "adam"
    model.compile( loss= loss, optimizer = optimizer, metrics = ["msle"])
    #training
    history = model.fit(train_x,train_y, epochs = num_epochs,batch_size= 70, verbose = 1,validation_data = (val_x,val_y))
    return history

def plotter(history,metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric,'val_'+metric])
    plt.show()
    



In [ ]:
#putting it all together

ann_without = build_net_no_dropout(X_train.shape[1])
hist_ann_without = model_run(ann_without,X_train,Y_test,Y_train,X_test,30)
plotter(hist_ann_without, "msle")
plotter(hist_ann_without, "loss")


In [ ]:
ann_with = build_net_with_dropout(X_train.shape[1])
hist_ann_with = model_run(ann_with,X_train,Y_test,Y_train,X_test,40)
plotter(hist_ann_with, "msle")
plotter(hist_ann_with, "loss")

In [ ]:

Predictions = ann_without.predict(X_test)
Real = Y_test
dft = pd.DataFrame({"real_2017":list(Y_test),"Prediction_2017":list(Predictions)})
plt.figure(figsize =(20,4))
plt.plot(dft["real_2017"])
plt.plot(dft["Prediction_2017"])
plt.legend(["Real", "Predictions"])
plt.show()

# Biomass  Supply chain Optimization

The goal of this section is to accurately model all components and relationships in the biomass to energy supply chain( site=> biomass depots=> Biomass refinery)and find the best configuration of the that maximizes the utilization of the of available outputs from the the biomass sites. To acheive this objective, we will create objects that model the  component(depots & refineries). In this section we will also format the problem of utilization optimization as a Reinforcement Learning problem and set up a Reinforcement learnning Environmentwhich models a certain configuration that a DQN agent can interact with.

In [9]:
# each additional component in the waste to energy supply chain will be represented 

class Component():
    def __init__(self, index, name, num, cap,k,pointx,refinery, data=biomass.copy(),distance = distance.copy(),spare= copy.deepcopy(points),sources=None):
        self.name = name
        self.index = index
        self.num = num       
        self.capacity = cap
        self.is_ref = refinery
        self.tree = rtree.index.Rtree()
        self.point = spare[index]
        
        self.pointx = copy.deepcopy(pointx)
            
    
        if refinery==True and sources!=  None:
            self.depot_left = sources.copy()
            for i,a in enumerate(sources):# sources should be index numbers
                p=spare[a]
                self.tree.insert(i,p+p,p)
                
        else:
            for i,p in enumerate(self.pointx):
                self.tree.insert(i,p+p,p)
            
       
        #print(len(self.pointx)) 
        
        # k nearest point
        self.sources= list(self.tree.nearest(self.point, k+1,objects='raw' ))
        if len(self.sources)>k+1:
            self.sources = self.sources[:k+1]
        
        self.sources = self.sources.copy()[1:]      
        #print(len(self.sources))
        self.sources_index = []
        if self.is_ref ==False:
            for i in self.sources:
                self.sources_index.append(spare.index(i))
            
        else: 
            for i in self.sources:
                self.sources_index.append(spare.index(i))
               
            
        #print("Original number of sources {}".format(len(self.sources)))
        
        #this part calculates the total output of the sources
        if self.is_ref==False:
            sources_output = data["2017"][self.sources_index].sum()
        
            self.under_util = 0
            if sources_output>self.capacity and self.is_ref == False:
                while True:
                    lee = data["2017"][self.sources_index].values
                    minx = np.argmin(lee, axis = 0)
                    self.sources.pop(minx)
                    self.sources_index.pop(minx)
                    sources_output = data["2017"][self.sources_index].sum()
                
                    if sources_output<=self.capacity:
                        self.under_util = self.capacity-sources_output
                        break
                    else:
                        continue
                
            if self.is_ref==False:    
                for i in self.sources:
                    self.pointx.remove(i)
                
            
                
            #print(len(self.pointx))        
            self.sources_output = sources_output 
        
         
    
        
        self.sources_index = [spare.index(i) for i in self.sources.copy()]
        self.total_dist = 0
        for i in self.sources_index:
            self.total_dist+=distance.iloc[i,self.index]
        
        #print("final number of sources {}".format(len(self.sources)))
        
    def prevent_duplicates(self):
        if self.is_ref ==True:
            depot_left = self.depot_left
            for iw in self.sources_index:
                depot_left.remove(iw)
            del(self.depot_left)
            return self.pointx,depot_left
        else:
            return self.pointx
            
    def add(self,df):
        q= int(self.index)
        df.Type[q] = str(self.name)
        return df
    def output(self):
        return self.sources_output
    def cost(self):
        if self.is_ref ==True:
            return self.total_dist
        else:
            return self.total_dist,self.under_util
    
            
        
        

def make_refinery(number,name, k, cap,depots,pointx,index=-1,refinery=True, allx=points.copy()):
    
    refineries = [] 
    if index==-1:        
        listr = [] 
        
        for i in range(number):
            index = allx.index(random.choice(pointx))
            
            if index not in listr :
                pointx.remove(allx[index])
                
                #print("current avail dept: {}".format(depots))
                comp = Component(index,name,i,cap,k,copy.deepcopy(pointx),refinery = refinery,sources=copy.deepcopy(depots))
                pointx,depots= comp.prevent_duplicates()
                refineries.append(comp)
                listr.append(index)
                
            else:
                i = i-1
                continue
    print("done")
    return  copy.deepcopy(refineries),copy.deepcopy(depots),copy.deepcopy(pointx)   

def make_components(number,name, k, cap,pointx,index=-1,refinery=False, allx = points.copy()):
    components = [] 
    if index==-1:        
        listr = [] 
        for i in range(number):
            index = allx.index(random.choice(pointx))
            if index not in listr:
                #print (pointx)
                pointx.remove(allx[index])
                comp = Component(index,name,i,cap,k,copy.deepcopy(pointx),refinery = refinery)
                pointx= comp.prevent_duplicates()
                components.append(comp)
                listr.append(index)
                
            else:
                i = i-1
                continue
    print("done")
    return copy.deepcopy(components),copy.deepcopy(pointx)
        
    
    
     
    
    



This custom reinforcement learning Environment has been engineered to incentivize actions that subsequently lead to lower costs(represented by the Evironment.state variable) and higher resource utilization. Each depot will be supplied by a maximum of 80 other biomass sites and has a capacity of 20,0000  while, each refinery can only be supplied by 5 depots and have a maximum capacity of 100,000. The Environment arrange ment is randomly initialized and so the initial values for cost and utilization will vary. I noticed that DQN agent performs significantly better in Environments with an initial  utilisation greater than or equal to 80%. Based on these finding to ensure best results I recomend that:
1. Only environments with initial utilization <80% be used to train the agent
2. Enronments that meet the above criterion should be picked as the model will only be able to optimize the exact enviromnment it was trained on.

In each step in this environment, the agent can select either a Depot or refinery and, move the selected component to a new unused site to the left or right. The dataset contains 2417 locations from which the optimal sites for the Refineries and Depots must be chosen.  


In [7]:
#creating a custom environment with gym

class Environment(Env):
    def __init__(self,num_dep,num_refinery, points = copy.deepcopy(points)):
        a = 0.001
        b = 1
        c = 1
        self.k = 80
        self.points = points 
        self.num_dep = num_dep
        self.refinery_cap = 100000
        self.num_refinery = num_refinery
        self.dep_cap = 20000
        #new
        
        self.depots,self.pointx = make_components(self.num_dep,"depot_location",self.k, self.dep_cap,self.points.copy())
        print(len(self.pointx))
        self.dep_points = [q.point for q in self.depots]
        self.dep_index =[points.index(q)for q in self.dep_points]
        #(number,name, k, cap,depots,pointx,index=-1,refinery=True, allx=points.copy())
        self.refineries,self.unused_depots,self.pointx = make_refinery(self.num_refinery,"refinery_location",5, 100000,copy.deepcopy(self.dep_index),copy.deepcopy(self.pointx))
        print(len(self.pointx)) 
        #new
        self.ref_points = [q.point for q in self.refineries]
        self.transport = 0
        self.underutil = 0
        self.ndim=(self.num_dep+self.num_refinery-1,2)
        self.observation_space = Box(high= np.inf,low = -np.inf,shape = (2,))
        self.action_space =Discrete(np.prod(self.ndim))
        all_dep_output = [q.sources_output for q in self.depots.copy()]
        all_dep_indexes = [q.index for q in self.depots.copy()]
        for i in self.depots:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
            
        for i in self.refineries:
            use = 0
            for b in i.sources_index:
                use+= all_dep_output[all_dep_indexes.index(b)]
            un = 100000-use
            tr = i.cost()
            self.transport+=tr
            self.underutil+=un
        self.state = (a*self.transport)+(c*self.underutil)
        self.episode_len = 200
        self.use = self.usage()
        self.previous_score = []
        self.reinitialize= {"depots":copy.deepcopy(self.depots),
                            "refinery":copy.deepcopy(self.refineries), 
                            "refinery_points":copy.deepcopy(self.ref_points),
                            "depot_points" :copy.deepcopy(self.dep_points),
                            "pointx":copy.deepcopy(self.pointx),
                            'Unused_depots':copy.deepcopy(self.unused_depots),
                            "usage" : copy.deepcopy(self.use),
                            "state": copy.deepcopy(self.state)
                           }
        print("utilizing {} percent of all available output".format(self.use))
        
        
    def usage(self):
        
        all_dep_output = [q.sources_output for q in self.depots]
        #print(all_dep_output)
        all_dep_indexes = [q.index for q in self.depots]
        all_ref_sources = [q.sources_index for q in self.refineries]
        all_ref_sources_flat=[]
        for e in all_ref_sources:
            for q in e:
                all_ref_sources_flat.append(q)
    
        
        total_usage = 0
        for i in all_ref_sources_flat:
            if i in all_dep_indexes:
                pla = all_dep_indexes.index(i)
                total_usage= total_usage+all_dep_output[pla]
                #print(total_usage)
        perc = 100*(total_usage/(biomass["2017"].sum()))
        return perc
         

    
            
    def move_depot(self, old_dep, new_dep_index, selection, attached_ref_pos, attached_ref):
        # Decompose and rebuild depot at new position
        self.pointx.extend(old_dep.sources + [old_dep.point])
        self.pointx.remove(self.points[new_dep_index])

        new_dep = Component(new_dep_index, "depot_location", old_dep.num, 20000, self.k, self.pointx.copy(), refinery=False)
        self.pointx = new_dep.prevent_duplicates()
        self.depots[selection] = new_dep

        # Update refinery with new depot source
        self.refineries[attached_ref_pos].sources_index.append(new_dep_index)
        self.refineries[attached_ref_pos].sources.append(new_dep.point)

        # Remove old depot from refinery
        self.refineries[attached_ref_pos].sources.remove(old_dep.point)
       # self.refineries[attached_ref_pos].sources_index.remove(old_dep.index)
        self.state = self.recalc()
        
            
        
        
    def observe(self):
        return np.array([self.state,self.use])
    
    def reset(self):
        self.depots = self.reinitialize["depots"]
        self.refineries = self.reinitialize["refinery"] 
        self.ref_points = self.reinitialize["refinery_points"]
        self.dep_points= self.reinitialize["depot_points"] 
        self.pointx=self.reinitialize["pointx"]
        self.unused_depots =self.reinitialize['Unused_depots']
        self.use = self.reinitialize["usage"]
        
        self.episode_len = 200
        self.previous_score = []
        self.state = self.recalc()
        obs = self.observe()
        return obs
        
    def recalc (self):
        a = 0.001
        b = 1
        c = 1
        self.transport = 0
        self.underutil = 0
        all_dep_output = [q.sources_output for q in self.depots.copy()]
        all_dep_indexes = [q.index for q in self.depots.copy()]
        for i in self.depots:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
        for i in self.refineries:
            use = 0
            for b in i.sources_index:
                use+= all_dep_output[all_dep_indexes.index(b)]
            un = 100000-use
            tr = i.cost()

            self.transport+=tr
            self.underutil+=un
        var = (a*self.transport)+(c*self.underutil)
        return var
        
    def close(self):
        pass

    def find_closest(self, target, less=True):
    # Ensure the list is sorted for binary search
        lst = [self.points.index(q) for q in copy.deepcopy(self.pointx)]
        lst.sort()

    # Find the insertion point for the target using bisect
        pos = bisect.bisect_left(lst, target)

    # If target is found in the list, return it
        if pos < len(lst) and lst[pos] == target:
            return lst[pos]

        if less:
        # Find the closest number less than the target
            if pos == 0:
                return None  # No number less than target
            else:
                return lst[pos - 1]  # Return the closest smaller number
        else:
        # Find the closest number greater than the target
            if pos == len(lst):
                return None  # No number greater than target
            else:
                return lst[pos]  # Return the closest larger number



        
    def step(self, action):
        reward = 0
        mapping = tuple(np.ndindex(self.ndim))
        action = mapping[action]
        self.episode_len -= 1 
        if action[0]>=(self.num_dep) and self.episode_len>0:
            # if the agent picks from [0, self.num_dep] that indicates a depot, else its a refinery
            # in this case it is a refinery that has been selected
            selection = action[0]-self.num_dep
            try:
                old_ref = self.refineries[selection]
            except IndexError:
                print(" refinery action number{} refinery list length{}".format(selection,len(self.refineries)))
                return IndexError
            old_ref_number = old_ref.num
            old_ref_index =old_ref.index
            if action[1] == 0:
                #moves left
                new_ref_index = self.find_closest(old_ref_index-10, less=True)
                if new_ref_index !=None:
                    self.refineries[selection] = None
                    #freeing up the old refinery's attached depots
                    for de in old_ref.sources_index:
                        self.unused_depots.append(de)
                    
                    #Component(index,name,i,cap,k,pointx.copy(),refinery = refinery,sources=depots.copy())
                    new_ref = Component(new_ref_index,"refinery_location",old_ref_number,100000,5,self.pointx,refinery=True,sources=self.unused_depots)
                    self.pointx.remove(new_ref.point)
                    self.pointx,self.unused_depots = new_ref.prevent_duplicates()
                    self.pointx.append(old_ref.point)
                    self.refineries[selection] =  new_ref
                    self.state= self.recalc()
                else:
                    reward-=4
                
                    
                
            elif action[1] ==1:
                #moves right
                new_ref_index = self.find_closest(old_ref_index+10, less=False)
                if new_ref_index !=None:
                    self.refineries[selection] = None
                    #freeing up the old refinery's attached depots
                    for de in old_ref.sources_index:
                        self.unused_depots.append(de)
                    
                    #Component(index,name,i,cap,k,pointx.copy(),refinery = refinery,sources=depots.copy())
                    new_ref = Component(new_ref_index,"refinery_location",old_ref_number,100000,5,self.pointx,refinery=True,sources=self.unused_depots)
                    self.pointx.remove(new_ref.point)
                    self.pointx,self.unused_depots = new_ref.prevent_duplicates()
                    self.pointx.append(old_ref.point)
                    self.refineries[selection] =  new_ref
                    self.state= self.recalc()
                else:
                    reward-=4

            
        
        elif action[0]>self.num_dep and self.episode_len>0:
            # 0 to num_dep-1 is a depot
            
            selection = action[0]
            try:
                old_dep = self.depots[selection]
            except IndexError:
                print("action depot{} depot list{}".format(selection,len(self.depots)))
                return IndexError
            old_dep_number = old_dep.num
            old_dep_index =old_dep.index
            attached_ref_pos = None
            attached_ref=None
            #checking if the depot is linked to any refinery no point moving a depot that is not summ
            for we in self.refineries:
                if old_dep_index in we.sources_index:
                    we.sources_index.remove(old_dep_index)
                    attached_ref_pos = self.refineries.index(we)
                    attached_ref = we 
                    break
                else:
                    continue
            if attached_ref !=None: # 
                if action[1] ==1: # right
                    new_dep_index = self.find_closest(old_dep_index+10, less=False)
                    if new_dep_index !=None:
                        self.move_depot(old_dep, new_dep_index, selection, attached_ref_pos, attached_ref)
                        #self.state = self.recalc()

                    else:
                        reward-6
        
                if action[1] == 0:#left 
                    new_dep_index = self.find_closest(old_dep_index-10, less=True)
                    if new_dep_index !=None:
                        self.move_depot(old_dep, new_dep_index, selection, attached_ref_pos, attached_ref)
                        #self.state = self.recalc()

                    else:
                        reward-=6
            else:
                reward-=6
                
            
            
        
        # Calculating the reward
        if len(self.previous_score)>0:
            reward += (30*(self.previous_score[-1]-self.state))/self.previous_score[-1]
            self.previous_score.append(self.state)
 
  
        elif len(self.previous_score)==0:
            self.previous_score.append(self.state) 
            
        # additional rewards
        use = self.usage()
        self.use = use
        if self.use>=80:
            reward+= 2*(self.use-80)/80
        else:
            reward+=2*(self.use-80)/80
            
        
        # Checking if episode is done
        if self.episode_len <= 0: 
            done = True
        else:
            done = False
        
        #Setting the placeholder for info
       
        info = {'usage':self.use,"cost":self.state}
        
        # Returning the step information
        return self.observe(), reward, done,info

Run the cell below, mutliple times until you get an environment that satisfies the criteria 

In [26]:
Enviro = Environment(25,5)
state = Enviro.observation_space.shape
actions = Enviro.action_space.n

done
586
done
581
utilizing 81.90351536764538 percent of all available output


In [27]:
[i.sources_index for i in Enviro.refineries]

[[267, 804, 676, 113, 816],
 [2194, 2220, 2008, 1761, 1495],
 [2282, 1520, 1394, 1684, 1209],
 [1226, 1163, 1139, 1800, 1089],
 [785, 710, 2051, 216]]

In [28]:
[i.index for i in Enviro.refineries]

[16, 2373, 1969, 2407, 2070]

In [ ]:
def random_steps_in_custom_env(env, num_steps):
    
    #state = env.reset()
    trajectory = []

    for _ in range(num_steps):
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)\
        print([w.index for w in env.refineries])
        trajectory.append((state, action, reward, next_state, done))
        state = env.reset() if done else next_state

    return trajectory


In [ ]:
random_steps_in_custom_env(Enviro, 100):
    

In [29]:
visualize_depots_refineries_px(Enviro.depots, Enviro.refineries, points2)

In [11]:
visualize_depots_refineries_px(Enviro.depots, Enviro.refineries, points2)

In [30]:
def save_config(name, env):
     with open(name, "wb") as file:
         pickle.dump(env,file)
         
def load_config (name):
    with open(name,"rb") as file:
        obj = pickle.load(file)
        return obj
    

In [31]:
save_config("80percentuse.pkl",Enviro)

# Model creation and DQN Training

In [15]:
# model
model = tf.keras.models.Sequential()    
model.add(tf.keras.layers.Dense(32, activation='tanh', input_shape =(1,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='tanh'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='tanh'))
model.add(tf.keras.layers.Dense(actions, activation='linear'))


In [33]:

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=200, target_model_update=1e-3)
    return dqn
Agent = build_agent(model, actions)
Agent.compile(tf.keras.optimizers.legacy.SGD(learning_rate=1e-3), metrics=['mae'])

In [ ]:


Agent.fit(Enviro , nb_steps=150000, visualize=False, verbose=1)

Agent.save_weights("optimizer.h5", overwrite = False)

# Perform the optimization after Training

For this I will be loading a saved initial configuration and a model that was trained on that environment.
Feel free to play around with this part 

In [ ]:
def optimize(model,env, steps,verbose= False):
    #model should be a dqn agent
    state =env.observe()
    env.episode_len = steps
    if env.episode_len>= steps:
        for i in range(steps):
            action = model.forward(state)  # Get the action from the trained agent
            next_state, reward, done, info = env.step(action)  # Apply action to environment
            if verbose:
                print("Action Taken:", action)
                print("Next State:", next_state)
                print("Reward:", reward)
                print("Info:", info)
    
            if done:
                print("Episode finished.")
                print("current utilization: {} current cost: {}".format(env.use,env.state))
                break
                
    
            state = next_state
        print("current utilization: {} current cost: {}".format(env.use,env.state))
    return env
            
            
         
    

In [16]:
 #a slight bug made my saved environment not fully revert to initial conditons
def Hard_env_reset(env):
    env.depots = copy.deepcopy(env.reinitialize["depots"])
    env.refineries = copy.deepcopy(env.reinitialize["refinery"] )
    env.ref_points = copy.deepcopy(env.reinitialize["refinery_points"])
    env.dep_points= copy.deepcopy(env.reinitialize["depot_points"])
    env.pointx= copy.deepcopy(env.reinitialize["pointx"])
    env.unused_depots = copy.deepcopy(env.reinitialize['Unused_depots'])
    env.use = copy.deepcopy(env.reinitialize["usage"])
    return env
        

In [17]:
env = load_config(r"/kaggle/input/initial-arrangment/initial_arangment.pkl")
env = Hard_env_reset(env)
env.observe()

array([1.18311021e+05, 8.06750093e+01])

In [24]:
#initial utilization
env.unused_depots = list(set(env.unused_depots ))

In [14]:
Enviro.unused_depots

[1739]

In [31]:
sum([i.output() for i in env.depots])/sum(biomass["2017"])

0.8366009054937635

In [36]:
len([i.index for i in env.depots])

25

In [38]:
[i.sources_index for i in env.refineries]

[[1525, 1525, 1525, 1525, 1525],
 [379, 379, 379, 379, 379],
 [1745, 1745, 1745, 1745, 1745],
 [1525, 1525, 1525, 1525, 1525],
 [2211, 1953, 2384, 1788]]

In [ ]:
#initial cost
env.recalc()

In [ ]:
visualize_depots_refineries_px(env.depots, env.refineries, points2)

In [32]:
#load model
Agent.load_weights("/kaggle/input/optimizer1/keras/default/1/shell_v1_optimizer_agent.h5")

NameError: name 'Agent' is not defined

In [ ]:
env =optimize(Agent,env,200)

In [ ]:
visualize_depots_refineries_px(env.depots, env.refineries, points2)

In [ ]:
save_config("bestcase.pkl",env)

# Visualization

In [ ]:
visualize_depots_refineries_px(Enviro.depots, Enviro.refineries, points2)